# Forecasting using naive models and ETS (from statsforecast)

Overall WindowAverage with window size of 21 / 28 performed the best, even compared to ML and SARIMA according to kaggle RMSE, followed by HoltWinters (ETS). We would recommend using these naive models as a very decent baseline regressor because it is computationally cheap to run (window average took about 1.5 mins to train on the entire train csv), and does not require much data manipulation before training.

Practically speaking, with a window average of 21, it means that to figure out how many items to stock the shops, FairStorage should take the average of the items sold in the past 3 weeks, and that would be a good minimum number to stock the shelves with.

Models used, along with kaggle score: 
- Naive (3.03)
- SeasonalNaive (2.73)
- WindowAverage (2.27)
- SeasonalWindowAverage (2.48)
- HoltWinters (2.29)

Additionally we tried just using the last 21 days of the train data as the forecast data, and it obtained a relatively decent score at 2.81. This was much better than SARIMA which were the first type of models that we tried doing.

In [2]:
TRAIN_CSV_PATH = r"../datasets/train.csv"
PRICES_CSV_PATH = r"../datasets/prices.csv"
CAL_CSV_PATH = r"../datasets/calendar.csv"
SAMPLE_CSV_PATH = r"../datasets/sample_submission.csv"

EXPORT_PATH = r"../submissions/"

In [3]:
import numpy as np
import pandas as pd

from statsforecast import StatsForecast
from statsforecast.models import Naive, SeasonalNaive, WindowAverage, SeasonalWindowAverage, HoltWinters

from sklearn.metrics import mean_squared_error

c:\Users\Tristan\Desktop\Assignments\ADS\ads-final-project\ads_env\lib\site-packages\statsforecast\core.py:21: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


### Data preparation, convering to relevant datatypes and preparing long form data for training models (one row is one item for one day)

In [4]:
df_train = pd.read_csv(TRAIN_CSV_PATH)
df_sample_sub = pd.read_csv(SAMPLE_CSV_PATH)
df_cal = pd.read_csv(CAL_CSV_PATH)

# Merge d to get dates
df_dates = pd.DataFrame(columns = ["d"], data = df_train.columns[6:])
df_dates = df_dates.merge(df_cal[["date", "d"]], on = "d", how = "left")

# Convert to appropriate datatypes
df_dates["d"] = df_dates["d"].astype("string")
df_dates["date"] = pd.to_datetime(df_dates["date"])

string_features = ["id", "item_id", "subcat_id", "category_id", "store_id", "region_id"]
for f in string_features :
    df_train[f] = df_train[f].astype("string")

df_cal["date"] = pd.to_datetime(df_cal["date"])
df_cal["weekday"] = df_cal["weekday"].astype("string")
df_cal["d"] = df_cal["d"].astype("string")
df_cal["wm_yr_wk"] = df_cal["wm_yr_wk"].astype(int)

In [5]:
def round_positive(s) :
    """
    Function to change values to 0 if negative, if not round to nearest int. Used for forecast dataframe that returns non-positive / floats.
    s: Pandas series
    """
    s[s < 0 ] = 0
    s = s.round().astype(int)

    return s

In [6]:
def print_rmse(y_true, y_pred) :
    """
    Function to print the rmse of a prediction. Accepts 2 pandas series.
    """
    print(np.sqrt(mean_squared_error(y_true, y_pred)))

In [7]:
# Function to convert predictions into submission csv
def convert_to_sub_csv(preds_df, method) :
    """
    Converts a forecast from the format given by statsforecast, to a submission format for kaggle. 
    preds_df: forecasted df straight from statsforecast output.
    method: statsforecast method used to generate forecasts (relevant column name of preds_df)
    """
    
    df_converted = preds_df[["unique_id", "ds", method]].pivot(index = "unique_id", columns = "ds", values = method)

    # Change col names back to day ints
    day_to_d = dict(zip(list(df_converted.columns), list(df_sample_sub.columns[1:])))
    df_converted = df_converted.rename(day_to_d, axis = 1).reset_index()

    # Round up to nearest int
    df_converted.iloc[:, 1:] = df_converted.iloc[:, 1:].round().astype(int)    

    # Sort into the original ordering by ID
    df_converted[["category", "store", "num", "region", "num_2"]] = df_converted["unique_id"].str.split("_", expand = True)
    df_converted["region"] = pd.Categorical(df_converted["region"], ["East", "Central", "West"])
    df_converted = df_converted.sort_values(by = ["region", "num_2", "category", "store", "num"])
    df_converted = df_converted.drop(["category", "store", "num", "region", "num_2"], axis =1)

    # Rename ID col
    df_converted = df_converted.rename(columns = {"unique_id" : "id"})

    return df_converted

In [8]:
dates_dict = dict(zip(list(df_dates["d"]), list(df_dates["date"])))
df_train_dates = df_train.rename(dates_dict, axis = 1)

df_naive_train = df_train_dates[["id"] + list(df_train_dates.columns[6:])].melt(id_vars = ["id"], var_name= "ds", value_name = "y")
df_naive_train = df_naive_train.rename(columns = {"id":"unique_id"})

### Forecasting with naive models

For each model, we recorded the performance using 2 methods, 
- the first was RMSE on a 80/20 split, 
- the second was training on the entire train.csv and producing a 21 day forecast, and capturing the RMSE from kaggle.

The train / test set split was from train.csv split at 2015-01-29, since estimated to provide a 80/20 split given the range of dates.

In [9]:
# Find 80% split point between the start and end points of train dataset
end_date = df_naive_train["ds"].iloc[-1] 
start_date = df_naive_train["ds"].iloc[0] 
split_date = (end_date - start_date) * 0.8 + start_date

# Split into train and validation sets
train = df_naive_train.loc[df_naive_train['ds'] < split_date]
valid = df_naive_train.loc[(df_naive_train['ds'] >= split_date)]

# Number of forecast steps for 20% forecast test
h = valid['ds'].nunique()

### Forecasting with WindowAverage (for window sizes of 12, 14, 21, 28)

Best model overall for our team, with window average = 21 the best performing (kaggle score = 2.27) across the different window sizes. We compared WindowAverage with other naive models such as naive, seasonal naive, and seasonal window average (predictions were compared using the Kaggle score). In the interests of keeping the notebook short, the code for these other models are not reproduced here. We only present the code for the best model.

WindowAverage might be the best compared to SARIMA or ML because it is computationally less intensive than the other methods, and makes it able to train on the entire dataset. 

However, not sure why WindowAverage outperforms the other seasonal models, when in fact we would expect the seasonal models to better capture the seasonality we detected in the EDA. This is worth further investigation.

Train on the entire train dataset and export for the next 21 days (uncomment lines to export csv)

NOTE: Takes about 6 mins to run the below chunk of code. 

In [11]:
wa_12 = StatsForecast(models=[WindowAverage(window_size=12)],
                      freq='D', n_jobs=-1)

wa_14 = StatsForecast(models=[WindowAverage(window_size=14)],
                      freq='D', n_jobs=-1)

wa_21 = StatsForecast(models=[WindowAverage(window_size=21)],
                      freq='D', n_jobs=-1)

wa_28 = StatsForecast(models=[WindowAverage(window_size=28)],
                      freq='D', n_jobs=-1)

In [20]:
wa_12_pred = wa_12.forecast(h=21, df = df_naive_train)
wa_12_pred = wa_12_pred.reset_index()
wa_12_pred = convert_to_sub_csv(wa_12_pred, "WindowAverage")
# wa_12_pred.to_csv(EXPORT_PATH+"/naive_models/wa_12.csv", header=True, index=False)

wa_14_pred = wa_14.forecast(h=21, df = df_naive_train)
wa_14_pred = wa_14_pred.reset_index()
wa_14_pred = convert_to_sub_csv(wa_14_pred, "WindowAverage")
# wa_14_pred.to_csv(EXPORT_PATH+"/naive_models/wa_14.csv", header=True, index=False)

wa_21_pred = wa_21.forecast(h=21, df = df_naive_train)
wa_21_pred = wa_21_pred.reset_index()
wa_21_pred = convert_to_sub_csv(wa_21_pred, "WindowAverage")
# wa_21_pred.to_csv(EXPORT_PATH+"/naive_models/wa_21.csv", header=True, index=False)

wa_28_pred = wa_28.forecast(h=21, df = df_naive_train)
wa_28_pred = wa_28_pred.reset_index()
wa_28_pred = convert_to_sub_csv(wa_28_pred, "WindowAverage")
# wa_28_pred.to_csv(EXPORT_PATH+"/naive_models/wa_28.csv", header=True, index=False)

Predict to compare RMSE on the 20% split

NOTE: Takes about 8m to run the code chunk below

In [12]:
df_wa_12 = wa_12.forecast(h = h, df = train)
df_wa_12 = df_wa_12.reset_index().merge(valid, on=['ds', 'unique_id'], how='left')

df_wa_14 = wa_14.forecast(h = h, df = train)
df_wa_14 = df_wa_14.reset_index().merge(valid, on=['ds', 'unique_id'], how='left')

df_wa_21 = wa_21.forecast(h = h, df = train)
df_wa_21 = df_wa_21.reset_index().merge(valid, on=['ds', 'unique_id'], how='left')

df_wa_28 = wa_28.forecast(h = h, df = train)
df_wa_28 = df_wa_28.reset_index().merge(valid, on=['ds', 'unique_id'], how='left')

We can see that RMSE gradually decreases with increasing window sizes. Testing on greater window sizes beyond 28 increases Kaggle RMSE (graph in presentation slide 20).
So the ideal window length is about 21 - 28.

In [17]:
print_rmse(df_wa_12["y"], df_wa_12["WindowAverage"])
print_rmse(df_wa_14["y"], df_wa_14["WindowAverage"])
print_rmse(df_wa_21["y"], df_wa_21["WindowAverage"])
print_rmse(df_wa_28["y"], df_wa_28["WindowAverage"])

2.777953544392085
2.7284115552768027
2.6655059744910456
2.6392974023341615


### ETS model: HoltWinters (season length = 7)
Performed second best overall compared to the rest of the modelling techniques (i.e. SARIMA, ML modelling) and WindowAverage

However, due to the computational cost, the training was done on only on 50% of the dataset.

Also tried other ETS models, such as simple exponential smoothing, and seasonal exp. smoothing, but they were not as good as HoltWinters.

In [18]:
# Subset the last 50% of the time series because of computation
df_naive_train = df_naive_train.set_index("ds")
df_naive_train = df_naive_train.sort_index()
df_naive_train["ds"] = df_naive_train.index

# Find midpoint
midpoint = start_date + ((end_date - start_date) / 2)

# Split into train and validation, from midpoint onwards
df_naive_train_subset = df_naive_train.loc[midpoint:]

# find date to split, 80% of subset
test_split_date = ((end_date - midpoint) * 0.8) + midpoint

# Reset index of train dataset
df_naive_train_subset = df_naive_train_subset.drop("ds", axis = 1)
df_naive_train_subset = df_naive_train_subset.reset_index()

# Split by date
train = df_naive_train_subset.loc[df_naive_train_subset['ds'] < test_split_date]
valid = df_naive_train_subset.loc[(df_naive_train_subset['ds'] >= test_split_date)]

h = valid['ds'].nunique()

We chose to use the last 20% of the 50% split to train HoltWinters because running on the entire 50% split took longer than 30 mins. 

NOTE: below chunk took 12.5 mins to run.

In [19]:
holt_winters = StatsForecast(models=[HoltWinters(season_length=7)],
                      freq='D', n_jobs=-1)

# Use the last 20% to train. 
holt_winters_pred = holt_winters.forecast(h = 21, df = valid)

In [20]:
holt_winters_pred = holt_winters_pred.reset_index()
holt_winters_sub = convert_to_sub_csv(holt_winters_pred, "HoltWinters")
# holt_winters_sub.to_csv("../submissions/naive_models/holt_winters.csv", index = False)

### Comparing other baseline models (not top submissions, so don't need to run the code below this)
With Naive, SeasonalNaive, WindowAverage, Seasonal Window Average models
- Testing for different seasons and window sizes (multiples of 7, because that was the seasonality we detected in the EDA)

Overall looking at the RMSE below, naive seemed to perform the best, followed by seasonal window average, then seasonal naive. 

It is strange that if naive method performed the best, we would presumse that seasonal naive would be better, or at least be 2nd place. More work could be done in EDA on the predictions to compare the trends and descriptive statistics to see whether the predictions are in line with the train set.

In [22]:
model = StatsForecast(models=[Naive(), 
                              SeasonalNaive(season_length=7), 
                              SeasonalWindowAverage(window_size=2, season_length=7)],
                      freq='D', n_jobs=-1)

p = model.fit_predict(h=h, df = train)
p = p.reset_index().merge(valid, on=['ds', 'unique_id'], how='left')
p.head()

,unique_id,ds,Naive,SeasonalNaive,SeasWA,y
0,Beauty_1_001_Central_1,2015-10-22,1.0,0.0,0.0,0
1,Beauty_1_001_Central_1,2015-10-23,1.0,0.0,0.5,0
2,Beauty_1_001_Central_1,2015-10-24,1.0,1.0,0.5,0
3,Beauty_1_001_Central_1,2015-10-25,1.0,0.0,0.5,0
4,Beauty_1_001_Central_1,2015-10-26,1.0,0.0,0.0,0


In [23]:
print(np.sqrt(mean_squared_error(valid["y"], p["Naive"])))
print(np.sqrt(mean_squared_error(valid["y"], p["SeasonalNaive"])))
print(np.sqrt(mean_squared_error(valid["y"], p["SeasWA"])))

4.5222606586103105
4.96096247060974
4.886322395297068


In [ ]:
print_rmse(p["y"], p["Naive"])
print_rmse(p["y"], p["SeasonalNaive"])
print_rmse(p["y"], p["SeasWA"])

In [24]:
model_2 = StatsForecast(models=[SeasonalNaive(season_length=14), 
                              SeasonalWindowAverage(window_size=2, season_length=14)],
                      freq='D', n_jobs=-1)

p_2 = model_2.fit_predict(h=h, df = train)
p_2 = p_2.reset_index().merge(valid, on=['ds', 'unique_id'], how='left')
p_2.head()

,unique_id,ds,SeasonalNaive,SeasWA,y
0,Beauty_1_001_Central_1,2015-10-22,0.0,0.0,0
1,Beauty_1_001_Central_1,2015-10-23,1.0,1.0,0
2,Beauty_1_001_Central_1,2015-10-24,0.0,0.5,0
3,Beauty_1_001_Central_1,2015-10-25,1.0,1.0,0
4,Beauty_1_001_Central_1,2015-10-26,0.0,0.0,0


In [25]:
print(np.sqrt(mean_squared_error(valid["y"], p_2["SeasonalNaive"])))
print(np.sqrt(mean_squared_error(valid["y"], p_2["SeasWA"])))

5.07329627993835
4.862353096961041


## Naive models: Conclusion and Future work 
WindowAverage (WA) models consistently performed the best compared to others, though with more data, HoltWinters may be able to surpass WindowAverage, as currently the HoltWinters model here is only using the last 10% (0.5 * 0.2 = 0.1) of train data to produce its 21 day forecast.

Though for some reason seasonal models performed worse compared to WA, the increasing performance by increasing window length by multiples of 7 until 21 - 28 suggests that some seasonality is still captured by the model.

Further work:
- Test the same models with differenced data. If we make time series stationary, perhaps window average would be even better because model does not account for seasonality. However, seasonal models may also perform better because even though the data could be differenced by 7, there could be some seasonality for the next 7 days which may be better captured by seasonal models. 
- Investigate why seasonal models did not perform better than non-seasonal models, even when accounting for different windows and season sizes. If the data was seasonal, we would assume that seasonal models would do better than non-seasonal models.
- EDA could be conducted to compare the variance and trending of the predictions given by each model, to assess if the predictions are following the trend of the existing data. Though a model may have a higher RMSE, if the trending / seasonality of the predicted data better fits the train data, perhaps it is an issue of using more data points to train the model rather than the model itself not being suitable for prediction.
- There were also some other naive models that we didn't try, such as historic average, or auto ETS. Auto ETS would be much more computationally intensive than WindowAverage, and as we have seen, even HoltWinters didn't give better performance than WindowAverage, even though it took more than 20 times longer to train.